In [1]:
import pandas as pd
import pyarrow.parquet as pa
import spacy


/Users/hilarywang/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
#imports the load dataset from the Hugging Face DataSet library
from datasets import load_dataset
#loads the first 10,0000 rows from Parquet files in the variable data
data = load_dataset("parquet", data_files="train-00000-of-00001-090b52ccb189d47a.parquet", split='train[:10000]')
data

/Users/hilarywang/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['text'],
    num_rows: 10000
})

In [3]:
import re
#checking if row is title 
def is_title(row):
    return len(row['text'].split()) <= 2
    

In [4]:
def separate_title_and_text(dataset):
    final_rows = []
    current_title = None
    accumulated_text = []

    for row in dataset:
        if is_title(row):  # Identify if the row is a title
            if current_title is not None:
                # Save the previous title and its accumulated text
                final_rows.append({
                    'title': current_title,
                    'text': ' '.join(accumulated_text)
                })
            # Update the current title and reset the accumulated text
            current_title = row['text']
            accumulated_text = []
        else:
            # If it's part of the text, accumulate it
            accumulated_text.append(row['text'])

    # Append the last title and its text if exists
    if current_title is not None:
        final_rows.append({
            'title': current_title,
            'text': ' '.join(accumulated_text)
        })

    return final_rows

wiki_data = separate_title_and_text(data)


In [5]:
# converts lists into Hugging Face Dataset
wiki_data=data.from_list(wiki_data)

In [6]:
# Convert wiki_data into an array of dictionaries and add in a unique id for each article
wiki_dictionaries = []
for i in range(len(wiki_data)):
    new_dict = {"id": i, "title": wiki_data['title'][i], "text": wiki_data['text'][i]}
    wiki_dictionaries.append(new_dict)

In [7]:
#converts the list of dictionaries into Hugging Face Dataset
wiki_data=wiki_data.from_list(wiki_dictionaries)

In [8]:
import tiktoken
tiktoken.encoding_for_model('gpt-3.5-turbo')

<Encoding 'cl100k_base'>

In [9]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [11]:
# Split text into 400-char chunks with 20-char overlap
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""], 
)

In [12]:
#data_files="train-00000-of-00001-090b52ccb189d47a.parquet"

In [13]:
data_files = wiki_data

In [14]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name = "sentence-transformers/all-mpnet-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

data_files.set_format("torch", columns=["text"])

def generate_embedding(batch):
    inputs = tokenizer(batch['text'], return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    batch_embeddings=embeddings.numpy().tolist()
    return batch

In [15]:
embeddings = data_files.map(generate_embedding,batched=True,batch_size=16)

Map: 100%|█████████████████████████████| 988/988 [04:26<00:00,  3.71 examples/s]


In [16]:
import pinecone

In [20]:
from pinecone import Pinecone, ServerlessSpec
import torch
from transformers import AutoTokenizer, AutoModel

# Initialize model
model_name = "sentence-transformers/all-mpnet-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Generate embeddings
def generate_embedding(batch):
    inputs = tokenizer(batch['text'], return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    # Return a dictionary with the embeddings
    return {"embedding": embeddings.numpy().tolist()}

# Generate embeddings in batches
data_with_embeddings = data_files.map(generate_embedding, batched=True, batch_size=16)
embeddings = data_with_embeddings['embedding']

# Initialize Pinecone
pc = Pinecone(api_key="188cbb39-7aad-4697-8c0f-d9172a679d56")
index_name = "quickstart"

# Get existing index or create new one
try:
    index = pc.Index(index_name)
except:
    pc.create_index(
        name=index_name,
        dimension=768,  # mpnet-base-v2 dimension is 768
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    index = pc.Index(index_name)

# Prepare and upload vectors in batches
batch_size = 100
for i in range(0, len(embeddings), batch_size):
    batch_vectors = []
    for j, e in enumerate(embeddings[i:i + batch_size]):
        batch_vectors.append({
            "id": str(i + j),
            "values": e,
        })
    
    # Upsert batch
    index.upsert(
        vectors=batch_vectors,
        namespace="ns1"
    )
    print(f"Uploaded batch {i//batch_size + 1}")

Map: 100%|█████████████████████████████| 988/988 [05:54<00:00,  2.79 examples/s]


PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sun, 17 Nov 2024 00:30:14 GMT', 'Content-Type': 'application/json', 'Content-Length': '100', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '1280', 'x-pinecone-request-id': '8126062859090472282', 'x-envoy-upstream-service-time': '40', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Vector dimension 768 does not match the dimension of the index 2","details":[]}


In [6]:
pc